In [189]:
import cadquery as cq
from jupyter_cadquery.cadquery import show, PartGroup, Part
from jupyter_cadquery import set_defaults, set_sidecar
from pathlib import Path

set_defaults(axes=True, axes0=True, grid=False, tools=False, theme="dark", default_edgecolor=(0,0,0))

In [204]:
# a set of clips to hold in-ear monitors to a guitar strap
# with a hook at the far end to hold the ear pieces

# clip measurements
length      = 25
width       = 8
thickness   = 2  # of clip arms
gap         = 4  # between clip arms

hook_width  = width / 2
height      = gap   * 2  # hook height
hook_curve  = [
    (0.0,          height/4.0, 0),
    (length/6.25,  height/1.3, 0),
    (length/3.125, height/1.0, 0),
    (length/2.0,   height/1.3, 0),
    (length/1.5,   height/2.5, 0),
    (length/1.25,  height/1.6, 0),
]

def process(name: str, render):
        """display and save object"""
        show(render)
        name = f"clip/{name}"
        cq.exporters.export(render, f'{name}.stl')
        cq.exporters.export(render, f'{name}.svg')

clip = (
        cq.Workplane('front')
        # draw the overall box shape
        .box(length, gap + thickness, width)
        # round the edges
        .edges("|Z and <X").fillet(thickness)
        # cut out a shell
        .faces("+X or -Z or +Z").shell(thickness)
)

process("clip_wire", (
        clip.faces(">Z").workplane()
        # the seam is 5 units from the edge
        .center(-5, (gap/2)+1)
        # add wire groove
        .circle(thickness/2).cutThruAll()
))

process("clip_hook", (
        clip.add(
            cq.Workplane("XZ")
            # create hook base
            .rect(thickness, width/2)
            # trace base shape over hook curve
            .sweep(cq.Workplane("XY").spline(hook_curve, includeCurrent=True))
            # place hook base at back of clip
            .translate((-length/2+thickness, gap/2+thickness+1,0))
        )
))


Renderer(camera=CombinedCamera(far=100.0, height=600.0, mode='orthographic', orthoFar=100.0, position=(1.0, 1.…

Renderer(camera=CombinedCamera(far=100.0, height=600.0, mode='orthographic', orthoFar=100.0, position=(1.0, 1.…